In [2]:
from mealpy.swarm_based import PSO
import os
import numpy as np
from mealpy import PermutationVar, BinaryVar, FloatVar

In [3]:
parameters = [
    {"epoch": 1000, "pop_size": 100},
]

In [4]:
def read_sets(file_path, pop_size, num_city):
    """
    Reads a file containing ranked sets and returns them as a list of lists.

    Parameters:
    - file_path (str): Path to the file containing the ranked sets.
    - pop_size (int): Expected population size (number of sets).
    - num_city (int): Expected number of cities (elements per set).

    Returns:
    - list: A list of ranked sets, where each set is a list of integers.

    Raises:
    - ValueError: If the file content does not match the expected dimensions.
    """
    sets = []
    with open(file_path, "r") as file:
        for line in file:
            set = list(map(int, line.strip().split()))
            sets.append(set)

    # Validate the dimensions of the ranked sets
    if len(sets) != pop_size:
        raise ValueError(
            f"Expected {pop_size} sets, but found {len(sets)} in the file."
        )

    for set in sets:
        if len(set) != num_city:
            raise ValueError(
                f"Each set must have {num_city} elements. Found a set with {len(set)} elements."
            )

    return np.array(sets)

In [5]:
# Number of repetitions
num_trials = 10

In [6]:
def save_history_file(model, file_dir, file_prefix):
    """
    :param model: The optimization model containing the history
    :param file_dir: The directory where the history file will be saved
    :param file_prefix: Prefix for the history filename
    """
    history_file = os.path.join(file_dir, f"{file_prefix}_history.txt")

    with open(history_file, "w") as f:

        # for epoch in enumerate(model.history.list_global_best):
        for epoch in range(0, 1000):
            # fitness = agent.target.fitness, Global Fitness: {fitness}
            # solution = agent.solution, Solution: {solution}

            # Fetch additional metrics
            diversity = model.history.list_diversity[epoch]
            exploration = model.history.list_exploration[epoch]
            exploitation = model.history.list_exploitation[epoch]
            local_fitness = model.history.list_current_best_fit[epoch]
            global_fitness = model.history.list_global_best_fit[epoch]
            runtime = model.history.list_epoch_time[epoch]

            f.write(
                f"Epoch: {epoch+1}, Local Fitness: {local_fitness}, Global Fitness: {global_fitness}, Diversity: {diversity}, \
Exploration: {exploration}, Exploitation: {exploitation}, Runtime: {runtime}\n")

    print(f"History saved to {history_file}")

# <span style="color:blue; font-weight:bold;">F12021</span>


In [7]:
from opfunu.cec_based.cec2021 import F12021

f1 = F12021(10, f_bias=0)

In [8]:
p1 = {
    "bounds": FloatVar(lb=f1.lb, ub=f1.ub),
    "obj_func": f1.evaluate,
    "minmax": "min",
    "name": "F1",
    "save_population": True,
}

In [9]:
# Create the problem instance
bounds = FloatVar(lb=f1.lb, ub=f1.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [10]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F12021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F12021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

2025/02/20 05:56:29 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2025/02/20 05:56:29 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 1, Current best: 5082428871.345251, Global best: 5082428871.345251, Runtime: 0.00978 seconds
2025/02/20 05:56:29 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 2, Current best: 4896027037.8712225, Global best: 4896027037.8712225, Runtime: 0.01058 seconds
2025/02/20 05:56:29 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 3, Current best: 1711817975.9216883, Global best: 1711817975.9216883, Runtime: 0.01214 seconds
2025/02/20 05:56:29 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 4, Current best: 1711817975.9216883, Global best: 1711817975.9216883, Runtime: 0.00945 seconds
2025/02/20 05:56:29 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 5, Current best: 1592056184.375367, Global best: 1592056184.3753

History saved to ..\history\PSO\CEC2021\F12021\MT19937\trial_1\e=1000p=100\e=1000p=100_history.txt
Trial 1 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 2213.81814440305
Best solution: {'float': array([-82.10561366, -70.43000677, -29.61006291, -58.32762421,
        22.09276686,  59.94989684,   3.02465065,  18.56405065,
        76.6788868 ,  -9.54173211])}


2025/02/20 05:56:44 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 10, Current best: 532466433.5343426, Global best: 532466433.5343426, Runtime: 0.01305 seconds
2025/02/20 05:56:44 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 11, Current best: 344185340.9282101, Global best: 344185340.9282101, Runtime: 0.01087 seconds
2025/02/20 05:56:44 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 12, Current best: 344185340.9282101, Global best: 344185340.9282101, Runtime: 0.00788 seconds
2025/02/20 05:56:44 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 13, Current best: 216620058.2564281, Global best: 216620058.2564281, Runtime: 0.00769 seconds
2025/02/20 05:56:44 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 14, Current best: 168936405.87210014, Global best: 168936405.87210014, Runtime: 0.01279 seconds
2025/02/20 05:56:44 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, E

History saved to ..\history\PSO\CEC2021\F12021\MT19937\trial_2\e=1000p=100\e=1000p=100_history.txt
Trial 2 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 29.43391758005997
Best solution: {'float': array([-58.29799936, -70.43009565, -29.61069512, -58.32608945,
        22.09066611,  59.93857363,  27.46628478,  18.5597277 ,
        76.68109481, -29.61680362])}


2025/02/20 05:56:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 13, Current best: 82481034.34133048, Global best: 82481034.34133048, Runtime: 0.01052 seconds
2025/02/20 05:56:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 14, Current best: 82481034.34133048, Global best: 82481034.34133048, Runtime: 0.00757 seconds
2025/02/20 05:56:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 15, Current best: 82481034.34133048, Global best: 82481034.34133048, Runtime: 0.00741 seconds
2025/02/20 05:56:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 16, Current best: 66871332.22713959, Global best: 66871332.22713959, Runtime: 0.00698 seconds
2025/02/20 05:56:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 17, Current best: 60585735.72100843, Global best: 60585735.72100843, Runtime: 0.00717 seconds
2025/02/20 05:56:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epo

...

History saved to ..\history\PSO\CEC2021\F12021\MT19937\trial_8\e=1000p=100\e=1000p=100_history.txt
Trial 8 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 8756.856333064321
Best solution: {'float': array([ -8.5631687 , -70.42198464, -29.60558122, -58.28818833,
        22.10463909,  59.92251889,  78.52436439,  18.53843607,
        76.68214382, -71.54882777])}


2025/02/20 05:58:06 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 11, Current best: 543259823.121074, Global best: 543259823.121074, Runtime: 0.00821 seconds
2025/02/20 05:58:06 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 12, Current best: 543259823.121074, Global best: 543259823.121074, Runtime: 0.00828 seconds
2025/02/20 05:58:06 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 13, Current best: 543259823.121074, Global best: 543259823.121074, Runtime: 0.00831 seconds
2025/02/20 05:58:06 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 14, Current best: 494216105.84133613, Global best: 494216105.84133613, Runtime: 0.00690 seconds
2025/02/20 05:58:06 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 15, Current best: 473717776.95147926, Global best: 473717776.95147926, Runtime: 0.00876 seconds
2025/02/20 05:58:06 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch

History saved to ..\history\PSO\CEC2021\F12021\MT19937\trial_9\e=1000p=100\e=1000p=100_history.txt
Trial 9 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 11202.175350710606
Best solution: {'float': array([  4.91009777, -70.42894643, -29.61022708, -58.32598497,
        22.10073166,  59.95244007,  92.38353966,  18.53503181,
        76.6896628 , -82.92522518])}


2025/02/20 05:58:17 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 12, Current best: 2014545637.2896075, Global best: 2014545637.2896075, Runtime: 0.00764 seconds
2025/02/20 05:58:17 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 13, Current best: 1351274055.7321906, Global best: 1351274055.7321906, Runtime: 0.00830 seconds
2025/02/20 05:58:17 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 14, Current best: 784018723.1223041, Global best: 784018723.1223041, Runtime: 0.00792 seconds
2025/02/20 05:58:17 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 15, Current best: 277750484.6965108, Global best: 277750484.6965108, Runtime: 0.00922 seconds
2025/02/20 05:58:17 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F1, Epoch: 16, Current best: 254120555.44319472, Global best: 254120555.44319472, Runtime: 0.00858 seconds
2025/02/20 05:58:17 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: F

History saved to ..\history\PSO\CEC2021\F12021\MT19937\trial_10\e=1000p=100\e=1000p=100_history.txt
Trial 10 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 3703.8993684998995
Best solution: {'float': array([-90.47258029, -70.42747579, -29.60915468, -58.32635097,
        22.09774329,  59.94862283,  -5.57859782,  18.56704275,
        76.6833236 ,  -2.48764419])}


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F12021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F12021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F12021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F12021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F12021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F12021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F22021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F22021

f2 = F22021(10, f_bias=0)

In [ ]:
p2 = {
    "bounds": FloatVar(lb=f2.lb, ub=f2.ub),
    "obj_func": f2.evaluate,
    "minmax": "min",
    "name": "F2",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f2.lb, ub=f2.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F22021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F22021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F22021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F22021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F22021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F22021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F22021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F22021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F32021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F32021

f3 = F32021(10, f_bias=0)

In [ ]:
p3 = {
    "bounds": FloatVar(lb=f3.lb, ub=f3.ub),
    "obj_func": f3.evaluate,
    "minmax": "min",
    "name": "F3",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f3.lb, ub=f3.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F32021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F32021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F32021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F32021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F32021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F32021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F32021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F32021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F42021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F42021

f4 = F42021(10, f_bias=0)

In [ ]:
p4 = {
    "bounds": FloatVar(lb=f4.lb, ub=f4.ub),
    "obj_func": f4.evaluate,
    "minmax": "min",
    "name": "F4",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f4.lb, ub=f4.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F42021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F42021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F42021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F42021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F42021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F42021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F42021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F42021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F52021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F52021

f5 = F52021(10, f_bias=0)

In [ ]:
p5 = {
    "bounds": FloatVar(lb=f5.lb, ub=f5.ub),
    "obj_func": f5.evaluate,
    "minmax": "min",
    "name": "F5",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f5.lb, ub=f5.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F52021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F52021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F52021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F52021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F52021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F52021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F52021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F52021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F62021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F62021

f6 = F62021(10, f_bias=0)

In [ ]:
p6 = {
    "bounds": FloatVar(lb=f6.lb, ub=f6.ub),
    "obj_func": f6.evaluate,
    "minmax": "min",
    "name": "F6",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f6.lb, ub=f6.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F62021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F62021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F62021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F62021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F62021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F62021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F62021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F62021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F72021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F72021

f7 = F72021(10, f_bias=0)

In [ ]:
p7 = {
    "bounds": FloatVar(lb=f7.lb, ub=f7.ub),
    "obj_func": f7.evaluate,
    "minmax": "min",
    "name": "F7",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f7.lb, ub=f7.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F72021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F72021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F72021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F72021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F72021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F72021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F72021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F72021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F82021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F82021

f8 = F82021(10, f_bias=0)

In [ ]:
p8 = {
    "bounds": FloatVar(lb=f8.lb, ub=f8.ub),
    "obj_func": f8.evaluate,
    "minmax": "min",
    "name": "F8",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f8.lb, ub=f8.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F82021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F82021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F82021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F82021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F82021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F82021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F82021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F82021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F92021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F92021

f9 = F92021(10, f_bias=0)

In [ ]:
p9 = {
    "bounds": FloatVar(lb=f9.lb, ub=f9.ub),
    "obj_func": f9.evaluate,
    "minmax": "min",
    "name": "F9",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f9.lb, ub=f9.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F92021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F92021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F92021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F92021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F92021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F92021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F92021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F92021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

# <span style="color:blue; font-weight:bold;">F102021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F102021

f10 = F102021(10, f_bias=0)

In [ ]:
p10 = {
    "bounds": FloatVar(lb=f10.lb, ub=f10.ub),
    "obj_func": f10.evaluate,
    "minmax": "min",
    "name": "F10",
    "save_population": True,
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f10.lb, ub=f10.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F102021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F102021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(
        base_starting_solution_dir, trial_name, "Mt19937.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist

    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=mt19937)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F102021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F102021", "QuasiRandom"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(
        base_starting_solution_dir, trial_name, "QuasiRandom.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist

    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=QuasiRandom)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F102021"
)
base_history_dir = os.path.join("..", "history", "PSO", "CEC2021", "F102021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")

    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist

    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=IBM)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")

#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join(
    "..", "starting_solution", "CEC2021", "F102021"
)
base_history_dir = os.path.join(
    "..", "history", "PSO", "CEC2021", "F102021", "BeamSplitter"
)

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(
        base_starting_solution_dir, trial_name, "BeamSplitter.txt"
    )

    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist

    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = PSO.OriginalPSO(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=BeamSplitter)

        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)

        # Print the best results for each run
        print(
            f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}"
        )
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")